In [19]:
import pandas as pd
from os import listdir
from os.path import isfile, exists
import tools

PATH = '../data/data-IA'

In [20]:
data = pd.read_excel('../data/database.xls', skiprows=[0,1,2,3,4,5,6,7,8,9,10])
data.rename(columns = {data.columns[0]: 'status', data.columns[3]: 'number', data.columns[2]: 'date', 'NOM;PRENOM;DDN': 'name', data.columns[10]: 'label'}, inplace=True)
data["status"] = 1
data['name'].ffill(inplace=True)
data['date'].ffill(inplace=True)
data.head()

,status,name,date,number,AOU (QUANT),UAOROT,UMMA,UAMEVAL,USUCACET,AOU (QUAL),label
0,1,3542_ERNDIM QLOU 2019;D;,01/07/2019 10:01,1900033730,-,-,-,-,-,-,-
1,1,3542_ERNDIM QLOU 2019;E;,01/07/2019 10:06,1900033760,-,-,-,-,-,-,-
2,1,3542_ERNDIM QLOU 2019;F;,01/07/2019 10:10,1900033770,-,-,-,-,-,-,-
3,1,"3542_ERNDIM,DPT 2021;A;",10/03/2021 08:03,2100210312,-,-,-,-,-,-,RAS
4,1,"3542_ERNDIM,DPT 2021;C;",10/03/2021 08:17,2100210394,-,-,-,-,-,30H2MEBut,-


In [21]:
# on élimine les calibrations
data = data[data["name"].str.contains('ERNDIM')==False]
data = data[data["name"].str.contains('Test')==False]

In [22]:
# si pas de tiret ni RAS -> anormal(0)
for i in range(4,10):
    print(data.columns[i])
    data.loc[data[data.columns[i]] != '-', 'status'] = 0
    data.loc[data[data.columns[i]] == 'RAS', 'status'] = 1

# pas d'évidence dans interpretation -> normal
data.loc[data[data.columns[10]].str.contains("Pas d'évidence")==False, 'status'] = 0

AOU (QUANT)
UAOROT
UMMA
UAMEVAL
USUCACET
AOU (QUAL)


In [23]:
normal = data.loc[data['status'] == 1]
normal

,status,name,date,number,AOU (QUANT),UAOROT,UMMA,UAMEVAL,USUCACET,AOU (QUAL),label
43,1,ABDI;ABD ALLAH;08/03/2011,12/10/2020 08:20,2000709604,RAS,-,-,-,-,RAS,Pas d'évidence de trouble d'ordre métabolique ...
44,1,ABDIRAHMAN MOHAMUD;MOHAMED;14/07/2018,08/06/2019 07:00,1900001336,RAS,-,-,-,-,RAS,Pas d'évidence de trouble d'ordre métabolique ...
45,1,ABDIRAHMAN MOHAMUD;MOHAMED;14/07/2018,11/06/2019 09:50,1900001506,RAS,-,-,-,-,RAS,Pas d'évidence de trouble d'ordre métabolique ...
46,1,ABDULLAH;AHMED NAJI ABDULLAH;15/11/2012,26/04/2021 10:40,2100363385,RAS,-,-,-,-,RAS,Pas d'évidence de trouble d'ordre métabolique ...
52,1,ADOLPHE;RAILEY;12/11/2013,29/11/2019 15:00,1900398354,RAS,-,-,-,-,RAS,Pas d'évidence de trouble d'ordre métabolique ...
...,...,...,...,...,...,...,...,...,...,...,...
1126,1,YERNAUX;NICOLAS;16/09/2011,15/12/2020 16:00,2000904612,RAS,-,-,-,-,RAS,Pas d'évidence de trouble d'ordre métabolique ...
1127,1,ZAGHBANI LAMZOURI;FARES;29/06/2016,16/03/2021 08:00,2100230614,RAS,-,-,-,-,RAS,Pas d'évidence de trouble d'ordre métabolique ...
1128,1,ZAID BOUTTEVILLE;THYMEO;27/04/2017,04/12/2020 08:00,2000866233,RAS,-,-,-,-,RAS,Pas d'évidence de trouble d'ordre métabolique ...
1130,1,ZALIMKHANOV;ASKER;13/09/2016,02/02/2021 10:45,2100100830,RAS,-,-,-,-,RAS,Pas d'évidence de trouble d'ordre métabolique ...


In [24]:
not_normal = data.loc[data['status'] == 0]
not_normal

,status,name,date,number,AOU (QUANT),UAOROT,UMMA,UAMEVAL,USUCACET,AOU (QUAL),label
42,0,ABDELHAK;SABRI;29/03/2004,26/02/2021 08:00,2100177136,RAS,-,-,-,-,Présence de métabolites d'origine médicamenteu...,Pas d'évidence de trouble d'ordre métabolique ...
47,0,ABRAHAM;VICTOR;21/12/2012,02/06/2020 17:00,2000357924,RAS,-,-,-,-,Présence de métabolites d'origine bactérienne.,Pas d'évidence de trouble d'ordre métabolique ...
48,0,ABULADZE;QETA;24/02/2018,12/09/2019 08:15,1900224545,RAS,-,-,-,-,Présence de pics inconnus (métabolites d'origi...,Pas d'évidence de trouble d'ordre métabolique ...
49,0,ACHOURI;ASSIA;21/05/2020,09/02/2021 08:00,2100118651,RAS,-,-,-,-,Présence de pics inconnus (métabolites d'origi...,Pas d'évidence de trouble d'ordre métabolique ...
50,0,ACHOURI;ASSIA;21/05/2020,22/03/2021 09:00,2100249564,RAS,1.77,-,-,-,Présence de pics inconnus (métabolites d'origi...,Pas d'évidence de trouble d'ordre métabolique ...
...,...,...,...,...,...,...,...,...,...,...,...
1140,0,ZITOUNI;ASSIA;28/12/2009,01/07/2020 15:30,2000434336,Méthylmalonate : 14631 mmol/mol de créat...,-,-,-,-,-,Suivi d'acidurie méthylmalonique.
1141,0,ZITOUNI;ASSIA;28/12/2009,09/09/2019 11:45,1900217391,p-OH-phényl-lactate et p-OH-Phényl-pyruvate mo...,-,-,-,-,RAS,Suivi d'acidurie méthylmalonique.
1142,0,ZITOUNI;ASSIA;28/12/2009,17/02/2020 12:30,2000106900,Méthylmalonate très élevé.\nUn peu de méthylci...,-,14397.44,-,-,RAS,Suivi d'acudurie méthylmalonique.
1143,0,ZITOUNI;ASSIA;28/12/2009,17/11/2020 16:45,2000817850,-,0.72,12372.8,-,-,-,Suivi d'acidurie méthylmalonique.


In [29]:
data = data[data["date"].str.contains('2019')]
len(data)

281

Format du nom d'un chromatogramme : <br>
[nom][première lettre du prenom] - [date en aaaammjj]-[type de fichier].csv <br>
Par exemple pour Pierre Dupont le 06/12/2020 les fichiers seront : dupontp-20201206-chromatogram.csv et dupontp-20201206-ms.csv

Mais dans le cas de nom en deux mot il peut être pris le premier ou le second sans vraiment de règle prédéfinie, il faut donc tester les deux cas.<br>
La taille maximale de la partie nom + prénom est de 10 caractères <br>
Dans le cas d'un prenom composé le premier prenom peut être mi en entier.<br>

Non gérés : 
- fautes d'orthographe entre le nom et la base de donnée (akesim -> akesbim ou alijaja -> aliaja)
- tirets manquant.
- heure indiqué dans le nom
- problèmes sur la date
- nom composé pas en entier (barbiersca)
- pas du tout de chromato
- demo n'a pas de chromato

In [26]:
from importlib import reload
reload(tools)
listFiles = []

count = 0
for i in range(len(data)):
    temp = data.iloc[i]
    name = temp['name'].split(';')  # nom;prénom;date_de_naissance
    name[0] = name[0].split(' ')  # cas d'un nom composé
    name[1] = name[1].split(' ')  # cas d'un prenom composé
    date = temp['date'][:-6].split('/')
    # premier cas
    temp = name[0][0].lower() + name[1][0][0].lower()[0:10]
    filename = temp + '-' + date[2] + date[1] + date[0] + tools.CHROM_EXT
    if not exists(PATH + '/' +filename):
        # si ce n'est pas le premier cas
        if len(name[0]) > 1 and len(name[0][1])>0:
            temp = name[0][0].lower() + name[0][1] + name[1][0][0].lower()
            filename = temp[0:10] + '-' + date[2] + date[1] + date[0] + tools.CHROM_EXT
        if not exists(PATH + '/' +filename):
            # si ce n'est aucun des deux premiers
            if len(name[0]) > 1 and len(name[0][1])>0:
                temp = name[0][0].lower() + name[0][1][0] + name[1][0][0].lower()
                filename = temp[0:10] + '-' + date[2] + date[1] + date[0] + tools.CHROM_EXT
            if not exists(PATH + '/' +filename):
                temp = name[0][0].lower() + name[1][0].lower()
                filename = temp[0:10] + '-' + date[2] + date[1] + date[0] + tools.CHROM_EXT
                if not exists(PATH + '/' +filename):
                    if len(name[1]) > 1 and len(name[1][1])>0:
                        temp = name[0][0].lower() + name[1][0][0].lower() + name[1][1][0].lower()
                        filename = temp[0:10] + '-' + date[2] + date[1] + date[0] + tools.CHROM_EXT
                    if not exists(PATH + '/' +filename):
                        # si ce n'est pas un cas connu
                        print("Pas de chromatogramme détecté : ", "\nnom : ", name, "date : ", date)
                        listFiles.append('')
                        continue
    # le fichier chromatogramme est trouvé on vérifie si il y a aussi le fichier ms
    msfilename = filename[0:-len(tools.CHROM_EXT)] + tools.MOL_EXT
    if not exists(PATH + '/' +msfilename):
        print("Pas de fichier ms détecté", msfilename)
        listFiles.append('')
        continue
    count += 1
    listFiles.append(filename[0:-len(tools.CHROM_EXT)])

data['file'] = listFiles
data = data.loc[data['file'] != '']
print(listFiles)
print(count, " sur 281 dans la base et sur 535 fichiers existant")
    

Pas de chromatogramme détecté :  
nom :  [['AKESBI'], ['MOHAMED'], '02/11/2015'] date :  ['23', '09', '2019']
Pas de chromatogramme détecté :  
nom :  [['AKIN'], ['MEHMET', 'ALI'], '17/10/2009'] date :  ['09', '09', '2019']
Pas de chromatogramme détecté :  
nom :  [['ALIJAJ'], ['ASERLA'], '09/06/2019'] date :  ['05', '11', '2019']
Pas de chromatogramme détecté :  
nom :  [['ALIJAJ'], ['ASERLA'], '09/06/2019'] date :  ['11', '09', '2019']
Pas de chromatogramme détecté :  
nom :  [['AOURDOU', 'NAIT', 'HADDOU'], ['YASSINE'], '17/10/2019'] date :  ['12', '12', '2019']
Pas de chromatogramme détecté :  
nom :  [['AUBERTIN'], ['JULIA'], '30/09/2019'] date :  ['04', '10', '2019']
Pas de chromatogramme détecté :  
nom :  [['BA'], ['MOUSA'], '17/12/2019'] date :  ['18', '12', '2019']
Pas de chromatogramme détecté :  
nom :  [['BA'], ['MOUSA'], '17/12/2019'] date :  ['18', '12', '2019']
Pas de chromatogramme détecté :  
nom :  [['BALOZIAN'], ['ARTUR'], '01/06/2018'] date :  ['27', '12', '2019']
P

In [28]:
save = data[["file", "status", "label"]]
save.to_csv("../data/database.csv", index=False)